# Sentence Transformers


Составление эмбедингов для вопроса пользователя и поиск наиболее близкого среди всего корпуса вопросов, релевантность определяется с помощью косинусной близости.

In [ ]:
!pip3 install -U sentence-transformers

In [24]:
import json
import numpy as np
import torch
import sys
import os

Так как все данные находятся в json файлах, а нам необходимо их иметь в быстро доступности, то напишем функцию, которая создает список из вопросов-ответов нашего датасета.

In [2]:
def extract_corpus_texts(json_path):

    with open(fr"{json_path}", "r") as read_file:
        texts = json.load(read_file)
        
    qa_sentences = set()
    paragraphs = texts["data"][0]["paragraphs"]
    
    for context in paragraphs:
        qas = context["qas"]
        for qa in qas:
            qa_sentences.add(qa["question"] + " " + qa["answers"][0]["text"])
    
    return list(qa_sentences)

## Embeddings with Sentence Transformers

[HuggingFace](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

In [3]:
from sentence_transformers import SentenceTransformer, util

model_sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

/Users/ruagyk6/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
question = "Не зачлись часы с карточки, хотя пришло сообщение от бота, что делать?"

In [6]:
emb1 = model_sentence_transformer.encode("Не зачислены часы в кабинет, с карточек, которые помечены как выполненные, что делать? обратитесь к оператору техподдержки")
emb2 = model_sentence_transformer.encode(question)
cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.7633]])


### Поиск близкой пары вопрос-ответ во всей базе знаний

**Compute cosine similarity between question and all corpus of texts**

In [7]:
train_texts = extract_corpus_texts("./train-v1.1.json")
test_texts = extract_corpus_texts("./dev-v1.1.json")
sentences = train_texts + test_texts
sentences[:10]

['нет доступа на добавление новых колонок и дорожек в wekan доске проекта Для решения Вашей проблемы, пожалуйста, обратитесь к оператору техподдержки в следующих каналах zulip: техническая поддержка миэм и проектный офис..',
 'по ошибке зачел лишние часы себе и александре. возможно ли как-то исправить или мне лучше потом в дальнейших карточках взаимозачет часов произвести? Удалить их не получится, можете сделать дальнейший взаимозачет часов.',
 'возникает ошибка при входе в проектный cabinet, что делать? Для решения Вашей проблемы, пожалуйста, обратитесь к оператору техподдержки в следующих каналах zulip: техническая поддержка миэм и проектный офис.',
 'в wekan не доступна кнопка добавления участников +. как исправить? Для решения Вашей проблемы, пожалуйста, обратитесь к оператору техподдержки в следующих каналах zulip: техническая поддержка миэм и проектный офис.',
 'долго висит комната в jitsi, вечная загрузка, что делать? Для решения Вашей проблемы, обратитесь в техподдержку, а такж

In [8]:
#Encode all sentences
embeddings = model_sentence_transformer.encode(sentences)
embebbidng_question = model_sentence_transformer.encode(question)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embebbidng_question, embeddings)

cos_sim 

tensor([[0.4574, 0.6394, 0.5116, 0.4688, 0.4505, 0.4290, 0.7476, 0.4809, 0.3390,
         0.5120, 0.3991, 0.5806, 0.4631, 0.4795, 0.6038, 0.5648, 0.4250, 0.6633,
         0.4187, 0.3746, 0.4358, 0.4763, 0.5856, 0.4976, 0.6770, 0.6643, 0.4871,
         0.6802, 0.4020, 0.6543, 0.5030, 0.5724, 0.4303, 0.3598, 0.4490, 0.5570,
         0.4520, 0.3732, 0.4154, 0.5904, 0.6124, 0.4562, 0.3688, 0.5018, 0.6541,
         0.5131, 0.4219, 0.7143, 0.3546, 0.4021, 0.3799, 0.2714, 0.3043, 0.2957,
         0.5439, 0.5583, 0.5634, 0.4650, 0.6496, 0.6416, 0.4203, 0.5465, 0.6141,
         0.6194, 0.4337, 0.3864, 0.6738, 0.5977, 0.5010, 0.5787, 0.4293, 0.4579,
         0.6528, 0.4830, 0.4715, 0.5386, 0.6156, 0.5509, 0.5090, 0.5805, 0.5532,
         0.4261, 0.4162, 0.2450, 0.4423, 0.6503, 0.4044, 0.6103, 0.5720, 0.4571,
         0.7075, 0.6886, 0.4038, 0.4797, 0.4774, 0.6014, 0.4754, 0.5535, 0.4627,
         0.5839, 0.3924, 0.4448, 0.6149, 0.6019, 0.4570, 0.4104, 0.5469, 0.6199,
         0.5389, 0.4764, 0.4

In [49]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

**Find top-1 embedding**

In [9]:
ind_max = cos_sim.argmax()
print(ind_max)
sentences[ind_max]

tensor(148)


'хочу продлить свой проект на следующий год, но не получается, что делать? Вам необходимо обратиться в проектный офис.'

**Find top-3 embedding**

In [11]:
index_top3 = np.argsort(cos_sim.numpy())[0][::-1][:3]
print(index_top3)
top3_embeddings = [sentences[index] for index in index_top3]
print(top3_embeddings)

[148   6 179]
['хочу продлить свой проект на следующий год, но не получается, что делать? Вам необходимо обратиться в проектный офис.', 'засчитали лишние часы, как исправить? Удалить их не получится, можете сделать дальнейший взаимозачет часов.', 'хочу удалить зачтенную карточку из wekan, что делать? Этого сделать нельзя, можно лишь исправить неправильно зачтенную карточку.']


***Напишем функции для поиска контекста по вопросу***

In [16]:
def find_relevant_context_cosine(sentence_transformer, question, sentences, top=1):
    #Encode all sentences
    embeddings = sentence_transformer.encode(sentences)
    embebbidng_question = sentence_transformer.encode(question)

    #Compute cosine similarity between all pairs
    cos_sim = util.cos_sim(embebbidng_question, embeddings)
        
    if top == 1:
        ind_max = cos_sim.argmax()
        cos_max = cos_sim.max()
        return cos_max, sentences[ind_max]
    else:
        # find top-n embedding
        index_top_n = np.argsort(cos_sim.numpy())[0][::-1][:top]
        cos_top_n = np.sort(cos_sim.numpy())[0][:top]
        return cos_top_n, [sentences[index] for index in index_top_n]


In [17]:
def find_relevant_context_semantic(sentence_transformer, question, sentences, top=1):
    #Encode all sentences
    embeddings = sentence_transformer.encode(sentences)
    embebbidng_question = sentence_transformer.encode(question)

    #Compute cosine similarity between all pairs
    relevant_texts = util.semantic_search(embebbidng_question, embeddings, top_k=top)[0]
    
    relevant_sentences = []
    for i in range(top):
        id = relevant_texts[i]["corpus_id"]
        relevant_sentences.append(sentences[id])
        
    return relevant_sentences

***Тестирование на пуле вопросов***

In [14]:
questions = [
    "Нет доступа к доске в векане, как исправить?",
    "Ошибка зачисления часов в кабинет?",
    "Некоторые участники проекта не видят доску в векане, что делать?",
    "Руководитель не может зачесть часы с карточек?",
    "Не отображается доска проекта в wekan",
    "Не могу зайти в викан, авторизовался через miem почту",
    "Надо добавить участников проекта на доску в векане, как это сделать?",
    "Не могу авторизоваться в гитлабе, выдает ошибку, как это исправить?",
    "Проблема с зачислением часов в статистику проекта в кабинете, что делать?",
    "Не пришли часы из тайги?",
    "Как решить проблему с двойными аккаунтами?",
    "Почему в статистике кабинета не отображаются коммиты?",
    "Не могу загрузить файл в репозиторий проекта?",
    "У моего проекта нет доски в векане, что делать?"
            ]

In [21]:
dict_words = {
    "wekan": ["векан", "викан", "викин", "wekan", "векэн", "WEKAN", "wkan","векана", "векане", "викана", "викане"],
    "gitlab": ["гитлаб", "гитлаба", "гитлабе", "гит", "гите", "гитхаб", "git", "gitlab", "GIT", "GITLAB"],
    "cabinet": ["кабинет", "кабинит", "cabinet", "CABINET", "кабинета", "кабинете"],
    "taiga": ["тайга", "taiga", "TAIGA"],
    "jitsi": ["джитси", "jitsi", "джисти", "JITSI"],
    "wiki": ["вики", "wiki", "WIKI"],
    "zulip": ["зулип", "zulip", "ZULIP","зулипа","зулипе","злипа"],
    "ВШЭ": ["вышка",  "ниу вшэ", "высшая школа экономики", "вше", "вшэ"],
    "miem": ["миэм", "МИЭМ", "миэма", "миэму", "мем", "мэм", "мием", "miem", "MIEM", "meim"],
    "ЕЛК": ["ЕЛК", "елк", "екл", "ЕЛКа"]
}

Проверим с помощью косинусной близости

In [33]:
sys.path.insert(1, "../preproccesing/parse_jsons.py")

In [34]:
from parse_jsons import replace_incorrect_spellings

In [42]:
import re
import pymorphy2

In [44]:
morph = pymorphy2.MorphAnalyzer()

In [47]:
for q in questions:
    for key, value in dict_words.items():
        if (key in q or key in replace_incorrect_spellings(q, dict_words, morph)) and key in ["wekan", "cabinet", "gitlab", "jitsi", "project", "taiga", "wiki"]:
            sentences = extract_corpus_texts(f"../data/handmade_dataset/format of deeppavlov/full/{key}.json")
        else:
            continue
    cos_value, context = find_relevant_context_cosine(model_sentence_transformer, q, sentences)
    if cos_value < 0.5:
        print("Пока что я только учусь, поэтому не могу ответить на Ваш вопрос. Пожалуйста, обратитесь в каналы технической поддержки в zulip.")
    else:
        print("Question:", q)
        print("Cosine value", cos_value)
        print("Relative context:", context.split("? ")[1], end="\n\n")

Question: Нет доступа к доске в векане, как исправить?
Cosine value tensor(0.8313)
Relative context: Можно Вас добавить иным способом в доску, обратитесь к оператору техподдержки

Question: Ошибка зачисления часов в кабинет?
Cosine value tensor(0.6507)
Relative context: Для решения Вашей проблемы, пожалуйста, обратитесь к оператору техподдержки в следующих каналах zulip: техническая поддержка миэм и проектный офис..

Question: Некоторые участники проекта не видят доску в векане, что делать?
Cosine value tensor(0.8453)
Relative context: Вас необходимо добавить на доску проекта с почты с доменом @edu.hse.ru

Question: Руководитель не может зачесть часы с карточек?
Cosine value tensor(0.7501)
Relative context: Этого сделать нельзя, можно лишь исправить неправильно зачтенную карточку.

Question: Не отображается доска проекта в wekan
Cosine value tensor(0.7444)
Relative context: Стараемся оперативно решить проблему.

Question: Не могу зайти в викан, авторизовался через miem почту
Cosine val

Посчитаем метрики оценив, на сколько вопросов модель ответила правильно

In [48]:
y_true = [1] * len(questions)
y_pred = [1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0]

In [51]:
print("F1-score:", f1_score(y_true, y_pred))
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Империческая:", sum(y_pred) / sum(y_true))

F1-score: 0.7272727272727273
Accuracy: 0.5714285714285714
Империческая: 0.5714285714285714


Проверим по семантической близости

In [56]:
for q in questions:
    for key, value in dict_words.items():
        if (key in q or key in replace_incorrect_spellings(q, dict_words, morph)) and key in ["wekan", "cabinet", "gitlab", "jitsi", "project", "taiga", "wiki"]:
            sentences = extract_corpus_texts(f"../data/handmade_dataset/format of deeppavlov/full/{key}.json")
        else:
            continue
    context = find_relevant_context_semantic(model_sentence_transformer, q, sentences)
    
    print("Question:", q)
    print("Relative context:", *context, end="\n\n")

Question: Нет доступа к доске в векане, как исправить?
Relative context: Не могу получить доступ к доске в викан, мой аккаунт её не видит, что делать? Можно Вас добавить иным способом в доску, обратитесь к оператору техподдержки

Question: Ошибка зачисления часов в кабинет?
Relative context: Различается количество часов в векане и кабинете, что делать? Для решения Вашей проблемы, пожалуйста, обратитесь к оператору техподдержки в следующих каналах zulip: техническая поддержка миэм и проектный офис..

Question: Некоторые участники проекта не видят доску в векане, что делать?
Relative context: У участника нет доступа к доске проекта ни с какого аккаунта, что делать? Вас необходимо добавить на доску проекта с почты с доменом @edu.hse.ru

Question: Руководитель не может зачесть часы с карточек?
Relative context: Руководитель хочет удалить зачтенную карточку, как исправить? Этого сделать нельзя, можно лишь исправить неправильно зачтенную карточку.

Question: Не отображается доска проекта в w

In [57]:
y_true = [1] * len(questions)
y_pred = [1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0]

print("F1-score:", f1_score(y_true, y_pred))
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Империческая:", sum(y_pred) / sum(y_true))

F1-score: 0.6666666666666666
Accuracy: 0.5
Империческая: 0.5


### Поиск близкой пары вопрос-ответ с помощью другой модели

Для данной модели также реализуем косинусный и семантический поиск

In [58]:
cpu_device = torch.device("cpu")

In [59]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('clips/mfaq').to(device=cpu_device)

/Users/ruagyk6/Library/Python/3.9/lib/python/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [60]:
question = "Не получается зайти в гитлаб, возможно ошибка из-за двойного аккаунта, что делать?"

In [61]:
query_embedding = model.encode(question)
corpus_embeddings = model.encode(sentences)
relevant_texts = util.semantic_search(query_embedding, corpus_embeddings, top_k=2)[0][0]
print(relevant_texts)

{'corpus_id': 0, 'score': 0.7097870111465454}


In [62]:
sentences[relevant_texts["corpus_id"]]

'Не могу зайти в векан, пишет ошибку 502? это технический сбой, который скоро должен исправиться.'

In [63]:
questions = [
    "Нет доступа к доске в векане, как исправить?",
    "Ошибка зачисления часов в кабинет?",
    "Некоторые участники проекта не видят доску в векане, что делать?",
    "Руководитель не может зачесть часы с карточек?",
    "Не отображается доска проекта в wekan",
    "Не могу зайти в викан, авторизовался через miem почту",
    "Надо добавить участников проекта на доску в векане, как это сделать?",
    "Не могу авторизоваться в гитлабе, выдает ошибку, как это исправить?",
    "Проблема с зачислением часов в статистику проекта в кабинете, что делать?",
    "Не пришли часы из тайги?",
    "Как решить проблему с двойными аккаунтами?",
    "Почему в статистике кабинета не отображаются коммиты?",
    "Не могу загрузить файл в репозиторий проекта?",
    "У моего проекта нет доски в векане, что делать?"
            ]

Косинусная близость

In [64]:
for q in questions:
    for key, value in dict_words.items():
        if (key in q or key in replace_incorrect_spellings(q, dict_words, morph)) and key in ["wekan", "cabinet", "gitlab", "jitsi", "project", "taiga", "wiki"]:
            sentences = extract_corpus_texts(f"../data/handmade_dataset/format of deeppavlov/full/{key}.json")
        else:
            continue
    cos_value, context = find_relevant_context_cosine(model_sentence_transformer, q, sentences)
    if cos_value < 0.5:
        print("Пока что я только учусь, поэтому не могу ответить на Ваш вопрос. Пожалуйста, обратитесь в каналы технической поддержки в zulip.")
    else:
        print("Question:", q)
        print("Cosine value", cos_value)
        print("Relative context:", context.split("? ")[1], end="\n\n")

Question: Нет доступа к доске в векане, как исправить?
Cosine value tensor(0.8313)
Relative context: Можно Вас добавить иным способом в доску, обратитесь к оператору техподдержки

Question: Ошибка зачисления часов в кабинет?
Cosine value tensor(0.6507)
Relative context: Для решения Вашей проблемы, пожалуйста, обратитесь к оператору техподдержки в следующих каналах zulip: техническая поддержка миэм и проектный офис..

Question: Некоторые участники проекта не видят доску в векане, что делать?
Cosine value tensor(0.8453)
Relative context: Вас необходимо добавить на доску проекта с почты с доменом @edu.hse.ru

Question: Руководитель не может зачесть часы с карточек?
Cosine value tensor(0.7501)
Relative context: Этого сделать нельзя, можно лишь исправить неправильно зачтенную карточку.

Question: Не отображается доска проекта в wekan
Cosine value tensor(0.7444)
Relative context: Стараемся оперативно решить проблему.

Question: Не могу зайти в викан, авторизовался через miem почту
Cosine val

Посмотрим метрики

In [65]:
y_true = [1] * len(questions)
y_pred = [1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0]

print("F1-score:", f1_score(y_true, y_pred))
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Империческая:", sum(y_pred) / sum(y_true))

F1-score: 0.6666666666666666
Accuracy: 0.5
Империческая: 0.5


Семантическая близость

In [22]:
for q in questions:
    context = find_relevant_context_semantic(model, q, sentences)
    print("Question:", q)
    print("Relative context:", context, end="\n\n")

Question: Нет доступа к доске в векане, как исправить?
Relative context: ['Нет доски моего проекта в векане, помогите исправить? Обратитесь к оператору техподдержки']

Question: Ошибка зачисления часов в кабинет?
Relative context: ['Не пришли часы с выполненных карточек, хотя есть комментарий от бота о зачислении часов в кабинет, как исправить эту ошибку? обратитесь к оператору техподдержки']

Question: Некоторые участники проекта не видят доску в векане, что делать?
Relative context: ['Не вижу доску проекта в векане, аккаунты miem и hse синхронизированы, что делать? обратитесь к оператору техподдержки']

Question: Руководитель не может зачесть часы с карточек?
Relative context: ['Я руководитель и не могу проставить часы в карточках на доске векан, что делать? обратитесь к оператору техподдержки']

Question: Не отображается доска проекта в wekan
Relative context: ['Векан пишет ошибку доска не найдена для моего проекта, причем иногда доска появлялась, что делать? Вам необходимо авторизо

## Embeddings with BERT

In [66]:
import nltk
import pymorphy2
from nltk.corpus import stopwords
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка стоп-слов на русском языке для фильтрации
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruagyk6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
# Загрузка предварительно обученной модели BERT
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [68]:
def preprocess_text(text):
    # Токенизация текста и удаление стоп-слов
    tokens = nltk.word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return " ".join(filtered_tokens)

def extract_keywords(question):
    # Морфологический анализ каждого слова в вопросе и извлечение основы
    morph = pymorphy2.MorphAnalyzer()
    words = nltk.word_tokenize(question.lower())
    base_forms = [morph.parse(word)[0].normal_form for word in words if word.isalnum()]
    
    # Удаление стоп-слов из основных форм
    keywords = [word for word in base_forms if word not in stop_words]
    
    return keywords



def find_most_similar_sentence(question, corpus, bert_tokenizer, bert_model):
    # Препроцессинг вопроса и корпуса текстов
    question = "[CLS] " + question + " [SEP]"
    tokenized_question = bert_tokenizer.tokenize(question)
    tokenized_corpus = ["[CLS] " + sentence + " [SEP]" for sentence in corpus]
    
    # Преобразование текста в индексы BERT-токенов
    indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_question)
    tokens_tensor = torch.tensor([indexed_tokens])
    
    # Получение эмбеддингов для вопроса
    with torch.no_grad():
        outputs = bert_model(tokens_tensor)
        question_embedding = torch.mean(outputs.last_hidden_state, dim=1).squeeze().numpy()
    
    # Вычисление косинусного сходства между вопросом и каждым предложением в корпусе
    similarities = []
    for sentence in tokenized_corpus:
        # Преобразование текста в индексы BERT-токенов
        indexed_tokens = bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(sentence))
        tokens_tensor = torch.tensor([indexed_tokens])
        
        # Получение эмбеддингов для предложения
        with torch.no_grad():
            outputs = bert_model(tokens_tensor)
            sentence_embedding = torch.mean(outputs.last_hidden_state, dim=1).squeeze().numpy()
        
        # Вычисление косинусного сходства между вопросом и текущим предложением
        similarity = cosine_similarity([question_embedding], [sentence_embedding])[0][0]
        similarities.append(similarity)
    
    # Нахождение индекса предложения с наибольшим значением косинусного сходства
    most_similar_index = similarities.index(max(similarities))
    
    return corpus[most_similar_index]

In [69]:
# Пример использования функций
question = "Не зачлись часы с карточки, что делать?"

# Извлечение ключевых слов из вопроса
keywords = extract_keywords(question)

# Поиск наиболее близкого по смыслу предложения в корпусе текстов
most_similar_sentence = find_most_similar_sentence(question, sentences, bert_tokenizer, bert_model)

print("Извлеченные ключевые слова из вопроса:", keywords)
print("Наиболее близкое по смыслу предложение в корпусе текстов:", most_similar_sentence)

Извлеченные ключевые слова из вопроса: ['зачесться', 'часы', 'карточка', 'делать']
Наиболее близкое по смыслу предложение в корпусе текстов: Весь день не работает wekan? Стараемся оперативно решить проблему.


In [70]:
for q in questions:
    for key, value in dict_words.items():
        if (key in q or key in replace_incorrect_spellings(q, dict_words, morph)) and key in ["wekan", "cabinet", "gitlab", "jitsi", "project", "taiga", "wiki"]:
            sentences = extract_corpus_texts(f"../data/handmade_dataset/format of deeppavlov/full/{key}.json")
        else:
            continue
    context = find_most_similar_sentence(q, sentences, bert_tokenizer, bert_model)

    print("Question:", q)
    print("Cosine value", cos_value)
    print("Relative context:", context.split("? ")[1], end="\n\n")

Question: Нет доступа к доске в векане, как исправить?
Cosine value tensor(0.8156)
Relative context: это технический сбой, который скоро должен исправиться.

Question: Ошибка зачисления часов в кабинет?
Cosine value tensor(0.8156)
Relative context: Удалить их не получится, можете сделать дальнейший взаимозачет часов.

Question: Некоторые участники проекта не видят доску в векане, что делать?
Cosine value tensor(0.8156)
Relative context: Необходимо получить статус 'Администратор'.

Question: Руководитель не может зачесть часы с карточек?
Cosine value tensor(0.8156)
Relative context: Стараемся оперативно решить проблему.

Question: Не отображается доска проекта в wekan
Cosine value tensor(0.8156)
Relative context: Стараемся оперативно решить проблему.

Question: Не могу зайти в викан, авторизовался через miem почту
Cosine value tensor(0.8156)
Relative context: Сейчас доска должна отображаться, проверьте, пожалуйста, и авторизуйтесь через ЕЛК.

Question: Надо добавить участников проекта н

Посмотрим метрики для BERT

In [71]:
y_true = [1] * len(questions)
y_pred = [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0]

print("F1-score:", f1_score(y_true, y_pred))
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Империческая:", sum(y_pred) / sum(y_true))

F1-score: 0.4444444444444445
Accuracy: 0.2857142857142857
Империческая: 0.2857142857142857


**Сравнение bert и sentenceTransformers**

Объективно, можно заметить, что поиск походящего контекст с помощь модели Bert выдает худший результат, чем SentenceTransformers. 

Кроме того, на такое же количество вопросов модель BERT обрабатывает в 3-4 раза медленнее.

## Useful links

- [Metrics of QA](https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html)
- [Semantic model](https://huggingface.co/clips/mfaq)
- [More popular semantic model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)
